# Building Agents with Ollama: A Step-by-Step Guide

In [161]:
%pip install termcolor

Note: you may need to restart the kernel to use updated packages.


In [162]:
from termcolor import colored
import operator
import json
import requests
import os

In [163]:
agent_system_prompt_template = """
Environment: ipython\n

Cutting Knowledge Date: December 2023
Today Date: July 2024

You have access to the following functions:

{tool_descriptions}
Think very carefully before calling functions.
If you choose to call a function ONLY reply in the JSON format with no prefix or suffix:

"tool_choice": "name_of_the_tool",
"tool_input": "inputs_to_the_tool"

- `tool_choice`: The name of the tool you want to use. It must be a tool from your toolbox 
                or "no tool" if you do not need to use a tool.
- `tool_input`: The specific inputs required for the selected tool. 
                If no tool, just provide a response to the query.

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line

You are a helpful Assistant.
"""

In [164]:
def setup_ollama_model(model, system_prompt, temperature=0, stop=None):
    """
    Sets up the Ollama model configuration.

    Parameters:
    model (str): The name of the model to use.
    system_prompt (str): The system prompt to use.
    temperature (float): The temperature setting for the model.
    stop (str): The stop token for the model.

    Returns:
    dict: Configuration for the Ollama model.
    """
    return {
        "model_endpoint": "http://localhost:11434/api/generate",
        "model": model,
        "system_prompt": system_prompt,
        "temperature": temperature,
        "headers": {"Content-Type": "application/json"},
        "stop": stop,
    }


# Example configuration
ollama_config = setup_ollama_model(
    model="llama3.1", system_prompt=agent_system_prompt_template
)

In [165]:
def basic_calculator(input_str):
    """
    Perform a numeric operation on two numbers based on the input string.

    Parameters:
    input_str (str): A JSON string representing a dictionary with keys:
        - 'num1' (int): The first number.
        - 'num2' (int): The second number.
        - 'operation' (str): The operation to perform. Supported operations are 'add', 'subtract',
                             'multiply', 'divide', 'floor_divide', 'modulus', 'power', 'lt',
                             'le', 'eq', 'ne', 'ge', 'gt'.

    Returns:
    str: The formatted result of the operation.

    Raises:
    Exception: If an error occurs during the operation (e.g., division by zero).
    ValueError: If an unsupported operation is requested or input is invalid.
    """
    # Clean and parse the input string
    try:
        # Replace single quotes with double quotes
        input_str_clean = input_str.replace("'", '"')
        # Remove any extraneous characters such as trailing quotes
        input_str_clean = input_str_clean.strip().strip('"')

        input_dict = json.loads(input_str_clean)
        num1 = input_dict["num1"]
        num2 = input_dict["num2"]
        operation = input_dict["operation"]
    except (json.JSONDecodeError, KeyError) as e:
        return str(e), "Invalid input format. Please provide a valid JSON string."

    # Define the supported operations
    operations = {
        "add": operator.add,
        "subtract": operator.sub,
        "multiply": operator.mul,
        "divide": operator.truediv,
        "floor_divide": operator.floordiv,
        "modulus": operator.mod,
        "power": operator.pow,
        "lt": operator.lt,
        "le": operator.le,
        "eq": operator.eq,
        "ne": operator.ne,
        "ge": operator.ge,
        "gt": operator.gt,
    }

    # Check if the operation is supported
    if operation in operations:
        try:
            # Perform the operation
            result = operations[operation](num1, num2)
            result_formatted = (
                f"\n\nThe answer is: {result}.\nCalculated with basic_calculator."
            )
            return result_formatted
        except Exception as e:
            return str(e), "\n\nError during operation execution."
    else:
        return "\n\nUnsupported operation. Please provide a valid operation."

In [166]:
def reverse_string(input_string):
    """
    Reverse the given string.

    Parameters:
    - input_string (str): The string to be reversed.

    Returns:
    str: The reversed string.
    """
    # Reverse the string using slicing
    reversed_string = input_string[::-1]

    reversed_string = f"The reversed string is: {reversed_string}\n\nExecuted using the reverse_string function."
    # print (f"DEBUG: reversed_string: {reversed_string}")
    return reversed_string

In [167]:
def prepare_tools(tools):
    """
    Prepares the tool descriptions.

    Parameters:
    tools (list): List of tool functions.

    Returns:
    str: JSON-like formatted string describing the tools.
    """
    tools_info = []

    for tool in tools:
        # Extract the tool's name and description
        name = tool.__name__
        docstring = (
            tool.__doc__.strip() if tool.__doc__ else "No description available."
        )

        # Split docstring into description and parameters parts
        parts = docstring.split("\n\n", 1)
        description = parts[0].split("\n")[
            0
        ]  # Take the first line as the main description
        parameters_section = parts[1] if len(parts) > 1 else ""

        param_dict = {}

        # Parse parameters section
        if "Parameters:" in parameters_section:
            parameters_lines = parameters_section.split("\n")
            param_section_started = False

            for line in parameters_lines:
                if "Parameters:" in line:
                    param_section_started = True
                    continue
                if not param_section_started or not line.strip():
                    continue

                if line.strip().startswith("-"):
                    # Likely a parameter line
                    param_name_type_desc = line.split(":", 1)
                    if len(param_name_type_desc) == 2:
                        param_name_type, description = param_name_type_desc
                        param_name = (
                            param_name_type.split("(", 1)[0].strip().lstrip("- ")
                        )
                        param_type = (
                            param_name_type.split("(", 1)[1].split(")", 1)[0]
                            if "(" in param_name_type
                            else "str"
                        )
                        param_dict[param_name] = {
                            "param_type": param_type.strip(),
                            "description": description.strip(),
                            "required": True,  # Assuming all parameters are required
                        }

        # Structure the tool info
        tool_info = {"name": name, "description": description, "parameters": param_dict}
        tools_info.append(tool_info)

    # Format the tools_info as a string
    tools_description = "\n".join(
        [
            f"Use the function '{info['name']}' to: {info['description']}\n{json.dumps(info, indent=2)}"
            for info in tools_info
        ]
    )

    return tools_description


tools = [basic_calculator, reverse_string]
tools_description = prepare_tools(tools)
print(tools_description)

Use the function 'basic_calculator' to:  The operation to perform. Supported operations are 'add', 'subtract',
{
  "name": "basic_calculator",
  "description": " The operation to perform. Supported operations are 'add', 'subtract',",
  "parameters": {
    "'num1'": {
      "param_type": "int",
      "description": "The first number.",
      "required": true
    },
    "'num2'": {
      "param_type": "int",
      "description": "The second number.",
      "required": true
    },
    "'operation'": {
      "param_type": "str",
      "description": "The operation to perform. Supported operations are 'add', 'subtract',",
      "required": true
    }
  }
}
Use the function 'reverse_string' to:  The string to be reversed.
{
  "name": "reverse_string",
  "description": " The string to be reversed.",
  "parameters": {
    "input_string": {
      "param_type": "str",
      "description": "The string to be reversed.",
      "required": true
    }
  }
}


In [168]:
def generate_response(prompt, config):
    """
    Generates a response from the Ollama model based on the provided prompt.

    Parameters:
    prompt (str): The user query.
    config (dict): The Ollama model configuration.

    Returns:
    dict: The response from the model.
    """
    agent_system_prompt = agent_system_prompt_template.format(
        tool_descriptions=tools_description
    )
    payload = {
        "model": config["model"],
        "format": "json",
        "prompt": prompt,
        "system": agent_system_prompt,
        "stream": False,
        "temperature": config["temperature"],
        "stop": config["stop"],
    }

    try:
        response = requests.post(
            config["model_endpoint"],
            headers=config["headers"],
            data=json.dumps(payload),
        )
        response_json = response.json()
        response_dict = json.loads(response_json["response"])

        print(f"\n\nResponse from Ollama model: {response_dict}")
        return response_dict
    except requests.RequestException as e:
        print(f"Error in invoking model! {str(e)}")
        return {"error": str(e)}

In [169]:
def execute_tool(response, tools):
    """
    Executes the appropriate tool based on the model's response.

    Parameters:
    response (dict): The model's response.
    tools (list): List of available tool functions.

    Returns:
    None
    """
    tool_choice = response.get("tool_choice")
    tool_input = response.get("tool_input")

    for tool in tools:
        if tool.__name__ == tool_choice:
            result = tool(tool_input)
            print(colored(result, "cyan"))
            return

    print(colored(tool_input, "cyan"))

In [170]:
prompt = "What is 5+5?"
response = generate_response(prompt, ollama_config)
execute_tool(response, tools)



Response from Ollama model: {'tool_choice': 'basic_calculator', 'tool_input': '{"num1": 5, "num2": 5, "operation": \'add\'}'}


The answer is: 10.
Calculated with basic_calculator.


In [171]:
prompt = "Reverse this string: I want to be reversed!"
response = generate_response(prompt, ollama_config)
execute_tool(response, tools)



Response from Ollama model: {'tool_choice': 'reverse_string', 'tool_input': 'I want to be reversed!'}
The reversed string is: !desrever eb ot tnaw I

 Executed using the reverse_string function.
